In [ ]:
import csv
import numpy as np
from numpy import loadtxt
import pandas as pd

In [ ]:
bus = 73
samples = 20000
level = 70

Bus_data = loadtxt('BusData.dat')
Gen_data = loadtxt('GenData.dat')
Bra_data = loadtxt('BranchData.dat')

Pg_data   = loadtxt(('Pg_Profile_%d.csv' %bus), delimiter=',')
PF_data   = loadtxt(('PF_Profile_%d.csv' %bus), delimiter=',')
PD_data   = loadtxt(('Pd_Profile_%d.csv' %bus), delimiter=',')
TC_data   = loadtxt(('TC_Profile_%d.csv' %bus), delimiter=',')

Pred_data = loadtxt(('Result/NN_%d_PF_Predict_%d.csv' %(level, bus)), delimiter=',')

In [ ]:
branch = Bra_data.shape[0]
index1 = int(0.8*samples)
index2 = int(0.1*samples)
index3 = index1 + index2

idxs = range(index3,samples)

In [ ]:
# Verify label between prediction and test data
Label = np.zeros([index2,branch])
PF_Percent = PF_data[idxs] / Bra_data[:,5]

# Label each branch as either 1, 2, or 3
Label = (np.abs(PF_Percent) >= level/100 ) * 1# + \
# np.all([(np.abs(PF_Percent) <= 0.95),(np.abs(PF_Percent) > 0.75)], axis = 0) * 2 + \
# (np.abs(PF_Percent) > 0.95) * 3

print(index2 * branch - np.sum(Label+1 == Pred_data))

In [ ]:
#Only monitor branch that is in category 2
cat2_rate = Bra_data[Pred_data[1,:] == 2,:]
cat2_rate[:,0] = range(cat2_rate.shape[0])
print(cat2_rate[:,[0, 1, 5]])

# #Set PF to line limit for category 3
# cat3_rate = Bra_data[Pred_data[1,:] == 3,:]
# cat3_rate[:,0] = range(cat3_rate.shape[0])
# print(cat3_rate[:,1])

load_profile = PD_data[index3 + 1,:]
a = np.sum(Pred_data == (2),axis = 1)

In [ ]:
from pyomo.environ import *

# instance the pyomo abstract model
model = AbstractModel()

### Set
model.BUS    = Set(initialize = Bus_data[:,0]-1)                        #k
model.BRANCH = Set(initialize = Bra_data[:,0]-1)                        #j
model.GEN    = Set(initialize = Gen_data[:,0]-1)                        #i

def cat2_num(model):
    return cat2_rate[:,0]
model.BRALIM2 = Set(initialize = cat2_num)     #l

# def cat3_num(model):
#     return cat3_rate[:,0]
# model.BRALIM3 = Set(initialize = cat3_num)     #p

### Param
#Bus param = k
model.bus_num  = Param(model.BUS, initialize = Bus_data[:,1]-1)

load_profile = PD_data[0,:]
def bus_Pd(model, k):
    return load_profile[int(k)]
model.bus_Pd   = Param(model.BUS, initialize = bus_Pd)#, mutable = True)

model.bus_type = Param(model.BUS, initialize = Bus_data[:,3])


#Gen param = i
model.gen_num  = Param(model.GEN, initialize = Gen_data[:,1]-1)

model.gen_bus  = Param(model.GEN, initialize = Gen_data[:,2]-1)

model.gen_Pmax = Param(model.GEN, initialize = Gen_data[:,3])#, mutable=True)

model.gen_Pmin = Param(model.GEN, initialize = Gen_data[:,4])#, mutable=True)

model.gen_C    = Param(model.GEN, initialize = Gen_data[:,5])



#Branch param = j
model.branch_num   = Param(model.BRANCH, initialize = Bra_data[:,1]-1)

model.branch_fbus  = Param(model.BRANCH, initialize = Bra_data[:,2]-1)

model.branch_tbus  = Param(model.BRANCH, initialize = Bra_data[:,3]-1)

model.branch_x     = Param(model.BRANCH, initialize = Bra_data[:,4])

#Branch Limit 2 param = l
def num_rateA_2(model, l):
    return cat2_rate[int(l),1]-1
model.num_rateA_2   = Param(model.BRALIM2, initialize = num_rateA_2)#, mutable=True)

def branch_rateA_2(model, l):
    return cat2_rate[int(l),5]
model.branch_rateA_2 = Param(model.BRALIM2, initialize = branch_rateA_2)#, mutable=True)

# #Branch Limit 3 param = p
# def num_rateA_3(model, p):
#     return cat3_rate[int(p),1]-1
# model.num_rateA_3   = Param(model.BRALIM3, initialize = num_rateA_3)#, mutable=True)

# def branch_rateA_3(model, p):
#     return cat3_rate[int(p),5]
# model.branch_rateA_3 = Param(model.BRALIM3, initialize = branch_rateA_3)#, mutable=True)

#base MVA
model.Base = Param(default=100.0)
# model.BigM = Param(default=10000000)
# model.Mu   = 0.00001

### Var
model.bus_angle  = Var(model.BUS)
model.gen_supply = Var(model.GEN)
model.line_flow  = Var(model.BRANCH)
# model.u          = Var(model.BRALIM3, within=Binary)
# model.Pk1        = Var(model.BRALIM3)
# model.Pk2        = Var(model.BRALIM3)

### Objective function
def objfunction(model,i):
    totalcost = sum(model.gen_supply[i]*model.gen_C[i] for i in model.GEN) #* model.BaseMVA
    return totalcost

model.cost = Objective(rule=objfunction, sense=minimize)


### constraint
#Power balance equation
def PowerBal(model,k):
    buspower = sum(model.line_flow[j] for j in model.BRANCH if model.branch_tbus[j] == k) - sum(model.line_flow[j] for j in model.BRANCH if model.branch_fbus[j] ==k) + sum(model.gen_supply[i] for i in model.GEN if model.gen_bus[i]==k) - model.bus_Pd[k]
    return buspower == 0
model.cons_PowerBal = Constraint(model.BUS,rule=PowerBal)

#power output limitation for generator
def PGenMax(model,i):
    return model.gen_supply[i] <= model.gen_Pmax[i]
model.cons_PGenMax = Constraint(model.GEN,rule=PGenMax)
def PGenMin(model,i):
    return model.gen_Pmin[i] <=  model.gen_supply[i]
model.cons_PGenMin = Constraint(model.GEN,rule=PGenMin)


# line flow equation
def lineflow(model,j):
    return model.line_flow[j] == model.Base * ((model.bus_angle[model.branch_fbus[j]]-model.bus_angle[model.branch_tbus[j]])/model.branch_x[j])
model.cons_lineflow = Constraint(model.BRANCH, rule=lineflow)


# thermal constraint for category 2
def ThermalMax2(model,l):
    return model.line_flow[model.num_rateA_2[l]] <= model.branch_rateA_2[l]
model.cons_ThermalMax2 = Constraint(model.BRALIM2, rule=ThermalMax2)
def ThermalMin2(model,l):
    return -model.branch_rateA_2[l] <=  model.line_flow[model.num_rateA_2[l]] 
model.cons_ThermalMin2 = Constraint(model.BRALIM2, rule=ThermalMin2)

# def ThermalMax3(model,p):
#     return model.line_flow[model.num_rateA_3[p]] <= model.branch_rateA_3[p]
# model.cons_ThermalMax3 = Constraint(model.BRALIM3, rule=ThermalMax3)
# def ThermalMin3(model,p):
#     return -model.branch_rateA_3[p] <=  model.line_flow[model.num_rateA_3[p]] 
# model.cons_ThermalMin3 = Constraint(model.BRALIM3, rule=ThermalMin3)

# # set thermal constraint for category 3
# def ThermalMax3(model,p):
#     return model.Pk1[p] == model.branch_rateA_3[p] * model.u[p]
# model.cons_ThermalMax3 = Constraint(model.BRALIM3, rule=ThermalMax3)
# def ThermalMin3(model,p):
#     return model.Pk2[p] == model.branch_rateA_3[p] * (model.u[p]-1)
# model.cons_ThermalMin3 = Constraint(model.BRALIM3, rule=ThermalMin3)
# def Thermal3(model,p):
#     return model.Pk1[p] + model.Pk2[p] == model.line_flow[model.num_rateA_3[p]]
# model.cons_Thermal3 = Constraint(model.BRALIM3, rule=Thermal3)


In [ ]:
# dcopf = model.create_instance()
# dcopf.cons_ThermalMax2.pprint()
# dcopf.pprint()


In [ ]:
### set the solver
solver = SolverFactory('solver\gurobi.exe')
dcopf = model.create_instance()
results = solver.solve(dcopf, tee=False)

### Suppress infeasible error warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

# Pd_List = np.zeros([index2,Bus_data.shape[0]])
# Pg_List = np.zeros([index2,Gen_data.shape[0]])
# PF_List = np.zeros([index2,Bra_data.shape[0]])
# TC_List = np.zeros([index2,1])
# Optimal = np.zeros([index2,1])


In [ ]:
import time

levels = np.array([ 70, 75, 80, 85, 90, 95 ]) 
nlevel = levels.shape[0]
run = 5
total_t = np.zeros(run)
run_t  = np.zeros((nlevel,run))

for z in range(nlevel):
    level = levels[z]
    Pred_data = loadtxt(('Result/NN_%d_PF_Predict_%d.csv' %(level, bus)), delimiter=',')
    for t in range(run):

        SampleT = np.zeros([index2])
        ### Run solver to generate data samples
        m = index3
        n = 0

        while m < samples:

            #Only monitor branch that is in category 2  
            cat2_rate = Bra_data[Pred_data[n,:] == 2,:]
            cat2_rate[:,0] = range(cat2_rate.shape[0])    
            # cat3_rate = Bra_data[Pred_data[n,:] == 3,:]
            # cat3_rate[:,0] = range(cat3_rate.shape[0])   

            load_profile = PD_data[m,:]

            #Solving the model and record time
            dcopf = model.create_instance()
            start_t = time.time()
            results = solver.solve(dcopf, tee=False)
            stop_t = time.time()  
            SampleT[n] = stop_t - start_t

            # if results.solver.termination_condition == "optimal":        
            #     Pg_List[n,:] = dcopf.gen_supply[:]()
            #     PF_List[n,:] = dcopf.line_flow[:]()
            #     TC_List[n,:] = dcopf.cost[:]()
            #     Optimal[n,:] = 1
            m = m+1
            n = n+1

            if m % 100 == 0:
                print(m)



        total_t[t] = np.sum(SampleT)
        #Total amount of time
        print('Run =', t)
        print('Total amount of time is', total_t[t])
    
    #Time for each level
    print(level)
    run_t[z] = total_t
    print('Average amount of time is', np.mean(total_t))

In [ ]:
pd_Time = pd.DataFrame(run_t)
pd_Time.to_excel('Result/Time_NN.xlsx')

In [ ]:
np.mean(run_t,axis = 1)